# Coronawiki dataset exploration

The purpose of the following notebook is to get familiar with the given Coronawiki data, as it is split among multiple files which serve different purposes.

As such, we will attempt to do the following tasks in this notebook:

- Preprocessing of the data,to make it more comfortable to use (Split the dataframes, give them another format, etc)
- Data wrangling: a lot of the data are timeseries which could be put together to derive interesting results
- First analysis phase

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

## Timeseries

The most important data we have in this dataset are time series of the Wikipedia views from 2018 to July 2020 for 14 different languages: one part are the total views for all of that language's wikipedia, a second part are the views for the articles that are related to Covid-19. Finally, we also have for the same window of time the views for different topics.

In [18]:
timeseries = pd.read_json("aggregated_timeseries.json.gz")
timeseries.head()

,ja.m,it,da.m,tr,no.m,en,sr,tr.m,en.m,no,...,ko.m,fi.m,sr.m,ja,fr,fi,ca,it.m,sv.m,ko
len,1197788,1594039,256451,346007,516838,6047509,632128,345790,6045654,531478,...,489181,480638,396063,1197856,2195949,481854,642031,1588312,1959446,490314
sum,"{'2018-01-01 00:00:00': 22328288, '2018-01-02 ...","{'2018-01-01 00:00:00': 3338750, '2018-01-02 0...","{'2018-01-01 00:00:00': 765123, '2018-01-02 00...","{'2018-01-01 00:00:00': 407629, '2018-01-02 00...","{'2018-01-01 00:00:00': 715031, '2018-01-02 00...","{'2018-01-01 00:00:00': 86763830, '2018-01-02 ...","{'2018-01-01 00:00:00': 192409, '2018-01-02 00...","{'2018-01-01 00:00:00': 493684, '2018-01-02 00...","{'2018-01-01 00:00:00': 135822131, '2018-01-02...","{'2018-01-01 00:00:00': 224417, '2018-01-02 00...",...,"{'2018-01-01 00:00:00': 1484496, '2018-01-02 0...","{'2018-01-01 00:00:00': 1319053, '2018-01-02 0...","{'2018-01-01 00:00:00': 451383, '2018-01-02 00...","{'2018-01-01 00:00:00': 7828155, '2018-01-02 0...","{'2018-01-01 00:00:00': 6441009, '2018-01-02 0...","{'2018-01-01 00:00:00': 523135, '2018-01-02 00...","{'2018-01-01 00:00:00': 111910, '2018-01-02 00...","{'2018-01-01 00:00:00': 12856884, '2018-01-02 ...","{'2018-01-01 00:00:00': 2383474, '2018-01-02 0...","{'2018-01-01 00:00:00': 819174, '2018-01-02 00..."
covid,"{'len': 30, 'sum': {'2018-01-01 00:00:00': 55,...","{'len': 33, 'sum': {'2018-01-01 00:00:00': 50,...","{'len': 4, 'sum': {'2018-01-01 00:00:00': 0, '...","{'len': 64, 'sum': {'2018-01-01 00:00:00': 1, ...","{'len': 10, 'sum': {'2018-01-01 00:00:00': 7, ...","{'len': 306, 'sum': {'2018-01-01 00:00:00': 57...","{'len': 9, 'sum': {'2018-01-01 00:00:00': 6, '...","{'len': 64, 'sum': {'2018-01-01 00:00:00': 3, ...","{'len': 306, 'sum': {'2018-01-01 00:00:00': 91...","{'len': 10, 'sum': {'2018-01-01 00:00:00': 2, ...",...,"{'len': 113, 'sum': {'2018-01-01 00:00:00': 6,...","{'len': 9, 'sum': {'2018-01-01 00:00:00': 0, '...","{'len': 9, 'sum': {'2018-01-01 00:00:00': 11, ...","{'len': 30, 'sum': {'2018-01-01 00:00:00': 26,...","{'len': 16, 'sum': {'2018-01-01 00:00:00': 62,...","{'len': 9, 'sum': {'2018-01-01 00:00:00': 2, '...","{'len': 49, 'sum': {'2018-01-01 00:00:00': 6, ...","{'len': 33, 'sum': {'2018-01-01 00:00:00': 139...","{'len': 8, 'sum': {'2018-01-01 00:00:00': 19, ...","{'len': 113, 'sum': {'2018-01-01 00:00:00': 3,..."
topics,{'Culture.Biography.Biography*': {'len': 14904...,{'Culture.Biography.Biography*': {'len': 29427...,{'Culture.Biography.Biography*': {'len': 57720...,{'Culture.Biography.Biography*': {'len': 70443...,{'Culture.Biography.Biography*': {'len': 11603...,{'Culture.Biography.Biography*': {'len': 14038...,{'Culture.Biography.Biography*': {'len': 37718...,{'Culture.Biography.Biography*': {'len': 70434...,{'Culture.Biography.Biography*': {'len': 14038...,{'Culture.Biography.Biography*': {'len': 11804...,...,{'Culture.Biography.Biography*': {'len': 75406...,{'Culture.Biography.Biography*': {'len': 10422...,{'Culture.Biography.Biography*': {'len': 37580...,{'Culture.Biography.Biography*': {'len': 14904...,{'Culture.Biography.Biography*': {'len': 38258...,{'Culture.Biography.Biography*': {'len': 10444...,{'Culture.Biography.Biography*': {'len': 10175...,{'Culture.Biography.Biography*': {'len': 29422...,{'Culture.Biography.Biography*': {'len': 14668...,{'Culture.Biography.Biography*': {'len': 75498...


In [17]:
timeseries.columns

Index(['ja.m', 'it', 'da.m', 'tr', 'no.m', 'en', 'sr', 'tr.m', 'en.m', 'no',
       'sv', 'nl.m', 'nl', 'da', 'de', 'fr.m', 'ca.m', 'de.m', 'ko.m', 'fi.m',
       'sr.m', 'ja', 'fr', 'fi', 'ca', 'it.m', 'sv.m', 'ko'],
      dtype='object')

Correspondence:
- ja -> Japanese
- it -> Italian
- da -> Danish
- tr -> Turkish?
- no -> Norwegian
- en -> English
- sr -> Serbian
- sv -> Swedish
- nl -> Dutch
- de -> German
- fr -> French
- ca -> Catalan?
- ko -> Korean
- fi -> Finnish

Not sure about the "?" ones. According to https://www.loc.gov/standards/iso639-2/php/langcodes-search.php, these correspond respectively to Turkish and Catalan.

### Splitting the timeseries data into different dataframes

As we can see, the data's format isn't ideal: for each language, the data is split into 3 Python dictionaries corresponding to the data described above, and it would be nice to separate these pieces of data to be able to read directly for each date, for example, the total number of views accross all languages, instead of having to iterate over each language's dictionnary every time.

This will also make the analysis phase easier later on.

### Total sum of views, views of articles related to Covid

In this part of the code we extract the three following kind of data, for each date:
- For every language's Wikipedia, the total number of views on that particular date
- For every language's Wikipedia, the total number of views for articles related to Covid-19 on that particular date
- For every language's Wikipedia, the percentage of views for articles related to Covid-19 on that particular date

Note that the two last dataframes might be redundant, but as we're given the data anyway, we choose to extract it after all.

---


Every resulting dataframe will have the following format:

 Column name          | Description                                                                                                                                                                                       |   |   |   |
|----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---|---|---|
| dates           | A particular date between January 2018 (inclusive) and July of 2020 (inclusive)                                                                                                                                             |   |   |   |
| language_code            | It can either be the total number of views for that language's Wikipedia, the number of views on Covid related articles on that same Wikipedia, or the percentage of these latter. There are 28 of these columns, as there are 14 languages and the data from desktop and mobile are separated.


---

We also extract another dataframe that simply maps for each language the number of articles that were considered in the original experiment.

In [ ]:
timeseries_total_sum_dict = {}
timeseries_covid_len_dict = {}
timeseries_covid_sum_dict = {}
timeseries_covid_percent_dict = {}
for cn in timeseries.columns:
    timeseries_total_sum_dict[cn] = timeseries[cn]['sum']
    timeseries_covid_len_dict[cn] = timeseries[cn]['covid']['len']
    timeseries_covid_sum_dict[cn] = timeseries[cn]['covid']['sum']
    timeseries_covid_percent_dict[cn] = timeseries[cn]['covid']['percent']

In [ ]:
sum_data_df = pd.DataFrame.from_dict(timeseries_total_sum_dict, orient = 'index').T
covid_len_data_df = pd.DataFrame.from_dict(timeseries_covid_len_dict, orient = 'index', columns = ['len']).T
covid_sum_data_df = pd.DataFrame.from_dict(timeseries_covid_sum_dict, orient = 'index').T
covid_percent_data_df = pd.DataFrame.from_dict(timeseries_covid_percent_dict, orient = 'index').T

In [ ]:
index_without_time = [x[:10] for x in covid_sum_data_df.index]
sum_data_df.index = covid_sum_data_df.index = covid_percent_data_df.index = pd.to_datetime(index_without_time)
sum_data_df['dates'] = covid_sum_data_df['dates'] = covid_percent_data_df['dates'] = sum_data_df.index
new_column_order = [sum_data_df.columns[-1]] + list(sum_data_df.columns[-2::-1])
sum_data_df = sum_data_df[new_column_order]
covid_sum_data_df = covid_sum_data_df[new_column_order]
covid_percent_data_df = covid_percent_data_df[new_column_order]

In [ ]:
sum_data_df.head()

In [ ]:
covid_sum_data_df.head()

In [ ]:
covid_percent_data_df.head()

### Checking for missing data

Before continuing further, let us check for missing data in the timeseries; this will help us avoid bad surprises later on.

In [ ]:
sum_data_df.isnull().any().any(), covid_sum_data_df.isnull().any().any(), covid_percent_data_df.isnull().any().any()

There appears to be some missing data in the percentage dataframe; let's check by language where that missing data is reported.

In [ ]:
missing_data_language = covid_percent_data_df.isnull().any(axis = 0)
missing_data_language[missing_data_language]

Looking at the paper, this corresponds to Swedish. Let's check in the other dataframes how that missing data manifests itself.

In [ ]:
sum_data_df.loc[:,['dates','sv']]

In [ ]:
covid_sum_data_df.loc[:,['dates', 'sv']]

In both dataframes, the missing data corresponds to 0 values for the dates. The NaN values thus correspond to a division by 0. Let us now check the first date where we get data for Swedish.

In [ ]:
swedish_mask = sum_data_df.loc[:,'sv'] > 0

In [ ]:
sum_data_df.loc[:,'sv'][swedish_mask]

In [ ]:
covid_sum_data_df.loc[:,'sv'][swedish_mask]

From what we can see, it appears that all the Swedish data from 2018 is missing; we will need to take that into consideration when doing our analysis.

The reason behind that is not clear: Wikipedia's swedish version has existed since 2001, and it's strange that the data from a whole year is either missing, or maybe it just hasn't been collected in the first place.

## Topics data

In [ ]:
country_to_topics = {}
for cn in timeseries.columns:
    country_to_topics[cn] = timeseries[cn]['topics']
topics_df = pd.DataFrame.from_dict(country_to_topics, orient = 'index')

In [ ]:
countries_to_topics_len = {}
countries_to_topics_sum = {}
countries_to_topics_percent = {}
for country in topics_df.index:
    countries_to_topics_len[country] = {}
    countries_to_topics_sum[country] = {}
    countries_to_topics_percent[country] = {}
    for topic in topics_df.columns:
        countries_to_topics_len[country][topic] = topics_df.loc[country,topic]['len']
        countries_to_topics_sum[country][topic] = topics_df.loc[country,topic]['sum']
        countries_to_topics_percent[country][topic] = topics_df.loc[country,topic]['percent']
countries_to_topics_len_df = pd.DataFrame.from_dict(countries_to_topics_len, orient = 'index')
countries_to_topics_sum_df = pd.DataFrame.from_dict(countries_to_topics_sum, orient = 'index')
countries_to_topics_percent_df = pd.DataFrame.from_dict(countries_to_topics_percent, orient = 'index')

In [ ]:
#countries_to_topics_sum_df.columns

In [ ]:
sum_environment_df = countries_to_topics_sum_df['STEM.Earth and environment']
percent_environment_df = countries_to_topics_percent_df['STEM.Earth and environment']
country_to_env_data_sum = {}
country_to_env_data_percent = {}
for country in sum_environment_df.index:
    country_to_env_data_sum[country] = sum_environment_df[country]
    country_to_env_data_percent[country] = percent_environment_df[country]
sum_environment_df = pd.DataFrame.from_dict(country_to_env_data_sum, orient = 'index').T
sum_environment_df.index = index_without_time
percent_environment_df = pd.DataFrame.from_dict(country_to_env_data_percent, orient = 'index').T
percent_environment_df.index = index_without_time

In [ ]:
sum_environment_df.head()

In [ ]:
percent_environment_df.head()

## Apple mobility

In [ ]:
apple_mobility = pd.read_csv("applemobilitytrends-2020-04-20.csv.gz")
#apple_mobility = apple_mobility.T
apple_mobility.head()

In [ ]:
apple_mobility.transportation_type.unique()

In [ ]:
apple_mobility_walking = apple_mobility[apple_mobility.transportation_type == 'walking']
apple_mobility_driving = apple_mobility[apple_mobility.transportation_type == 'driving']
apple_mobility_transit = apple_mobility[apple_mobility.transportation_type == 'transit']

In [ ]:
apple_mobility_walking_albania = apple_mobility_walking[apple_mobility_walking['region'] == 'Albania']

In [ ]:
timeseries_walking_albanian_data = apple_mobility_walking_albania[apple_mobility_walking_albania.columns[3:]]

In [ ]:
timeseries_walking_albanian_data.columns = pd.to_datetime(timeseries_walking_albanian_data.columns)

In [ ]:
#timeseries_albanian_data = timeseries_albanian_data.T

In [ ]:
timeseries_walking_albanian_data

In [ ]:
fig = plt.figure()
g = sns.lineplot( x = timeseries_walking_albanian_data.columns, y = timeseries_walking_albanian_data.iloc[0])
g.xaxis.set_major_locator(ticker.LinearLocator(7))
plt.xticks(fontsize=8)
g.set(xlabel='Dates', ylabel='Percentage')
g.set(title='Albanian walking')

## Global mobility

In [ ]:
global_mobility_report = pd.read_csv("Global_Mobility_Report.csv.gz")
global_mobility_report

## Interventions

In [ ]:
interventions = pd.read_csv("interventions.csv")
interventions

## Topics

In [ ]:
topics_linked = pd.read_csv("topics_linked.csv.xz")
topics_linked